## Reload Test Data 
Re loadd the data we need for the competition. We are using the Makorwiz Max Sharpe ration methods. 


In [13]:
import pandas as pd
import numpy as np
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
from datetime import datetime

# Define the downloader
def dataloader(stock_name,
              start = '1995-01-01 01:00:00',
              end = '2018-12-31 04:00:00',verbose = True):

  my_share = share.Share(stock_name)
  symbol_data = None

  try:
      symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,30,
                                            share.FREQUENCY_TYPE_DAY,1)
      if verbose:
        print(stock_name+" has been downloaded")
      
  except YahooFinanceError as e:
      print(e.message)
      sys.exit(1)
  df = pd.DataFrame(symbol_data)
  
  df['timestamp'] = [datetime.fromtimestamp(t/1000) for t in df['timestamp']]

  # test = df[(df['timestamp'] >= '2016-1-1 01:00:00') & (df['timestamp'] <= '2018-12-31 04:00:00')]
  return df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]

# Running the downloader
dat_comp_train = pd.read_csv("./data/train.csv")
stock_lst = [s.strip("\n") for s in list(dat_comp_train.keys() )]
stock_lst.pop(0)
stock_lst.pop()
test = {}
for i,stock_name in enumerate(stock_lst,0):
    # if i> 2: break
    try:
        df = dataloader(stock_name,start = '2018-01-01 01:00:00', end = '2018-07-16 04:00:00',verbose = False).copy()
        print(stock_name,i+1, len(df))
        if len(df) == 134:
            test[str(stock_name)] = list(df['close'])
    except:
        print(stock_name, i+1, "Failed")

pd.DataFrame(test).to_csv("./data/test.csv", index = False)


A  1 134
AAL  2 134
AAP  3 134
AAPL  4 134
ABBV  5 134
ABC  6 134
ABMD  7 134
ABT  8 134
ACN  9 134
ADBE  10 134
ADI  11 134
ADM  12 134
ADP  13 134
ADS  14 134
ADSK  15 134
AEE  16 134
AEP  17 134
AES  18 134
AFL  19 134
AGN  20 134
AIG  21 134
AIV  22 134
AIZ  23 134
AJG  24 134
AKAM  25 134
ALB  26 134
ALGN  27 134
ALK  28 134
ALL  29 134
ALLE  30 134
ALXN  31 134
AMAT  32 134
AMCR  33 134
AMD  34 134
AME  35 134
AMG  36 134
AMGN  37 134
AMP  38 134
AMT  39 134
AMZN  40 134
ANET  41 134
ANSS  42 134
ANTM  43 134
AON  44 134
AOS  45 134
APA  46 134
APD  47 134
APH  48 134
APTV  49 134
ARE  50 134
ARNC  51 134
ATO  52 134
ATVI  53 134
AVB  54 134
AVGO  55 134
AVY  56 134
AWK  57 134
AXP  58 134
AZO  59 134
BA  60 134
BAC  61 134
BAX  62 134
BBT  63 134
BBY  64 134
BDX  65 134
BEN  66 134
BF/B  67 Failed
BIIB  68 134
BK  69 134
BKNG  70 134
BKR  71 84
BLK  72 134
BLL  73 134
BMY  74 134
BR  75 134
BRK/B  76 Failed
BSX  77 134
BWA  78 134
BXP  79 134
C  80 134
CAG  81 134
CAH  82 134
CA

The 2<sup>nd</sup> hack: distribute money over all the positive stock on that day and sell short on every stock

In [0]:
dat = pd.read_csv("./data/test.csv")
target = pd.read_csv("./data/sample_submission.csv")
target.drop(columns=['Predicted'], inplace= True)
res = ( (dat - dat.shift(periods= 1 , fill_value= 0))/dat )

id_lst,pred_lst = [], []

for i, row in res.iterrows():
    if i == 0: continue
    row_lst = list(row)
    name_lst = list(row.keys() )

    S_pos = sum(r for r in row_lst if r>0)
    S_neg = sum(r for r in row_lst if r<0)

    id_lst += [str(i-1)+'_'+name for name in name_lst]
    pred_lst += [r/S_pos if r >0 else r/abs(S_neg) for r in row_lst]
    # pred_lst += [r/S_pos if r >0 else 0 for r in row_lst]
target2 = pd.DataFrame({'Id':id_lst, 'Predicted':pred_lst})
target = target.merge(target2, on = "Id",how = 'left')
target.fillna(0, inplace= True)
target['Id'] = target['Id'].apply(lambda id: id.split(" ")[0])
target.to_csv("submission_2.csv",index = False)

## Main Methods

In [14]:
dat = pd.read_csv("./data/test.csv")

In [47]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd


# conic optimization of price change covariance to weights using SLSQP method
def optimize(num_stocks, cov, expected_val, bounds):
    init_guess = np.ones(num_stocks) * (1.0 / num_stocks)
    weights = minimize(lambda w: (np.matmul(w.T,np.matmul(cov,w)) - np.matmul(w.T,expected_val)), init_guess,
                       method='SLSQP', options={'disp': False}, constraints=({'type': 'eq', 'fun': lambda w: 1.0 - np.sum(w)}), bounds=bounds)
    return weights.x


# multiprocessing approach to optimization (each row takes 30 seconds to optimize, so multithreading is crucial)
# modifies w_ret
def handle_optimization(i, bounds, lookback_window, prices, num_stocks, num_days):
    # get lookback of k days from real prices and lookahead of 1
    train = prices.iloc[i-(lookback_window-1):i+1]
    print(train.shape)
    print("optimizing row {}/{}...".format(i-(lookback_window-1), num_days - (lookback_window-1)))
    cov = train.cov()
    expected_val = train.mean()
    test = optimize(num_stocks, cov.values, expected_val, bounds)
    return (i-(lookback_window-1), test)


In [48]:
# input_path = 'results.csv' # full path to price change csv
input_path = './data/test.csv' # full path to price change csv
output_path = 'weights.csv' # full output path for weights csv
weight_min = -0.5 # minimum weights range bound
weight_max = 2 # maximum weights range bound
lookback_window = 5 # lookback window in days for optimization
num_threads = 4 # number of concurrent threads to use (upper bounded by available cpus)
target_days = 30 # number of desired output rows

In [50]:
import pandas as pd
import numpy as np
from scipy.optimize import Bounds
from functools import partial
import argparse
import multiprocessing as mp

num_threads = min(num_threads, mp.cpu_count())

###
# This conic optimizer is used to optimize a matrix of price changes into a matrix of portfolio weights in a range.
# We only want to optimize weights for the new predicted days in the conic optimizer.
# For each row the optimizer looks at today and a window of k days in the past
###

prices = pd.read_csv(input_path, index_col=0)

# consider only final target_days + the lookback window
prices = prices[-(target_days + (lookback_window - 1)):]

num_days = prices.shape[0]
num_stocks = prices.shape[1]

thread_pool = mp.Pool(num_threads) # concurrency

w_ret = np.zeros((target_days, prices.shape[1]))
print('output shape: {}'.format(w_ret.shape))
# bounds for weights. -.5/2 default
bounds = Bounds([weight_min]*num_stocks, [weight_max]*num_stocks)

# each row can take 30-60 seconds to optimize, so multithreading can be helpful
multithread_partial = partial(handle_optimization,
        bounds=bounds,
        lookback_window=lookback_window,
        prices=prices,
        num_stocks=num_stocks,
        num_days=num_days)

optimal_weights_unsorted = thread_pool.map(multithread_partial, range((lookback_window-1), num_days))

# realign output into matrix by indices
for i, w in sorted(optimal_weights_unsorted, key=lambda pair: pair[0]):
    w_ret[i, :] = w

print('Saving optimal weights to "{}"'.format(output_path))
pd.DataFrame(w_ret).to_csv(output_path)

KeyError: -34